In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [0]:
data_dir = ''

In [0]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [0]:
#Define transforms for the training data and testing data
train_transforms = transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

test_transforms = transforms.Compose([
                       transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225]),
                   ])

#pass transform here-in
train_data = ImageFolderWithPaths(data_dir + 'TRAIN_DATA', transform=train_transforms)
test_data = ImageFolderWithPaths(data_dir + 'TEST_DATA', transform=test_transforms)

#data loaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=8, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=8, shuffle=True)



In [0]:
print("Classes: ")
class_names = train_data.classes
print(class_names)

In [0]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title=[class_names[x] for x in classes])

# Get a batch of training data
inputs, classes,paths = next(iter(trainloader))
show_databatch(inputs, classes)

In [0]:
# Load the pretrained model from pytorch
dense121 = models.densenet121(pretrained=True)
print(dense121 )
print('Output Layer of Densenet121 : ', dense121 .classifier.out_features) # 1000 

In [0]:
print(dense121.classifier)

In [0]:
num_features = dense121.classifier.in_features
features = list(dense121.classifier.children())[:-1] # Remove last layer
print(features)

In [0]:
# Freeze training for all layers
for param in dense121.features.parameters():
    param.require_grad = False

In [0]:
features.extend([nn.Linear(num_features, len(class_names))])

In [0]:
dense121.classifier = nn.Sequential(*features)
print(dense121)

In [0]:
Epochs = 100
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(dense121.parameters(), lr=0.001, momentum=0.9)

In [0]:
from tqdm import tqdm

#if you have gpu then you need to convert the network and data to cuda
#the easiest way is to first check for device and then convert network and data to device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dense121.to(device)

dense121.train()

In [0]:

for epoch in range(Epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    pbar = tqdm(enumerate(trainloader))
    for i, data in pbar:
        # get the inputs
        inputs, labels,paths = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()
        # In PyTorch, we need to set the gradients to zero before starting to do backpropragation 
        # because PyTorch accumulates the gradients on subsequent backward passes. 
        # This is convenient while training RNNs. 
        # So, the default action is to accumulate the gradients on every loss.backward() call

        # forward + backward + optimize
        outputs = dense121(inputs)               #----> forward pass
        loss = criterion(outputs, labels)   #----> compute loss
        loss.backward()                     #----> backward pass
        optimizer.step()                    #----> weights update

        # print statistics
        running_loss += loss.item()
        
    torch.save(dense121.state_dict(), 'checkpoint_'+str(epoch)+'.pth')
    

print('Finished Training')

In [0]:
#Evaluation
dense121.load_state_dict(torch.load('checkpoint_99.pth'))
dense121.eval()

In [0]:
dataiter = iter(testloader)
images, labels, paths = dataiter.next()
show_databatch(images, labels)

In [0]:
images, labels = images.to(device), labels.to(device) #-->convert test image to cuda (if available)
outputs = dense121(images)                               #--> forward pass
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % class_names[predicted[j]]
                              for j in range(len(images))))
print('Ground Truth: ', ' '.join('%5s' % class_names[labels[j]]
                              for j in range(len(images))))

In [0]:
mid_pred =[]
high_pred =[]
low_pred =[]

batch_size =8
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels,paths = data
        #print(paths)
        images, labels = images.to(device), labels.to(device)
        outputs = dense121(images)
        _, predicted = torch.max(outputs.data, 1)
        for k in range(batch_size):
          if(predicted[k]==0):
            mid_pred.append(paths[k])
          if(predicted[k]==1):
            high_pred.append(paths[k])
          if(predicted[k]==2):
            low_pred.append(paths[k])
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
       

print('Accuracy of the network on the 528 test images: %d %%' % (
    100 * correct / total))

In [0]:
nb_classes = 3

confusion_matrix = torch.zeros(nb_classes, nb_classes)
with torch.no_grad():
    for i, (inputs, classes,paths) in enumerate(testloader):
        inputs = inputs.to(device)
        classes = classes.to(device)
        outputs = dense121(inputs)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(classes.view(-1), preds.view(-1)):
                confusion_matrix[t.long(), p.long()] += 1

print(confusion_matrix)